In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import xgboost as xgb
from statsmodels.tsa.statespace.sarimax import SARIMAX
from prophet import Prophet

import pickle
import os

In [2]:
# Load the dataset
file_name = "retail_store_inventory.csv"
df = pd.read_csv(file_name)

# ➡️ Convert the date column to datetime objects
df['Date'] = pd.to_datetime(df['Date'])

# ➡️ The target variable for forecasting is 'Units Sold'
TARGET_COLUMN = 'Units Sold'

print("Data Info:")
df.info()
print("\nFirst 5 Rows:")
print(df.head())

Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73100 entries, 0 to 73099
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Date                73100 non-null  datetime64[ns]
 1   Store ID            73100 non-null  object        
 2   Product ID          73100 non-null  object        
 3   Category            73100 non-null  object        
 4   Region              73100 non-null  object        
 5   Inventory Level     73100 non-null  int64         
 6   Units Sold          73100 non-null  int64         
 7   Units Ordered       73100 non-null  int64         
 8   Demand Forecast     73100 non-null  float64       
 9   Price               73100 non-null  float64       
 10  Discount            73100 non-null  int64         
 11  Weather Condition   73100 non-null  object        
 12  Holiday/Promotion   73100 non-null  int64         
 13  Competitor Pricing  73100 non-null 

In [3]:
# 1. Aggregate to Total Daily Sales
# We sum the 'Units Sold' across all stores and products for each day.
daily_sales = df.groupby('Date')[TARGET_COLUMN].sum().to_frame()

# 2. Prepare Exogenous Features (Daily Averages/Maximums)
# We aggregate the external features to the daily level, taking the mean or mode.

# Numerical Features (use mean)
exog_numerical = df.groupby('Date')[['Price', 'Discount', 'Competitor Pricing', 'Demand Forecast']].mean()

# Categorical Features (use mode - most frequent)
exog_categorical = df.groupby('Date')[['Weather Condition', 'Holiday/Promotion']].agg(lambda x: x.mode()[0])

# Combine all features
df_agg = daily_sales.join([exog_numerical, exog_categorical])
df_agg.index.name = 'Date' # Rename index for clarity

# ⚠️ Handle Missing Values (Check if any NaN resulted from aggregation)
# For this dataset, we expect very few, if any, NaN values after daily aggregation.
df_agg.fillna(method='ffill', inplace=True)

print("Aggregated Daily Data Head:")
print(df_agg.head().to_markdown(numalign="left", stralign="left"))

Aggregated Daily Data Head:
| Date                | Units Sold   | Price   | Discount   | Competitor Pricing   | Demand Forecast   | Weather Condition   | Holiday/Promotion   |
|:--------------------|:-------------|:--------|:-----------|:---------------------|:------------------|:--------------------|:--------------------|
| 2022-01-01 00:00:00 | 14484        | 57.5157 | 10.65      | 58.1831              | 150.313           | Cloudy              | 1                   |
| 2022-01-02 00:00:00 | 13415        | 60.6365 | 9.8        | 60.2371              | 139.107           | Cloudy              | 0                   |
| 2022-01-03 00:00:00 | 13681        | 56.7993 | 8.85       | 56.8813              | 141.853           | Cloudy              | 0                   |
| 2022-01-04 00:00:00 | 14084        | 52.993  | 10.6       | 52.8153              | 146.091           | Sunny               | 1                   |
| 2022-01-05 00:00:00 | 12572        | 55.9958 | 10.4       | 56.0426         

/tmp/ipython-input-3987031774.py:20: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_agg.fillna(method='ffill', inplace=True)


In [4]:
def create_time_series_features(data):
    # 1. Date Features
    data['dayofweek'] = data.index.dayofweek
    data['dayofyear'] = data.index.dayofyear
    data['weekofyear'] = data.index.isocalendar().week.astype(int)
    data['month'] = data.index.month
    data['year'] = data.index.year

    # 2. Lag Features (Target Variable Lags)
    # Sales from 1, 7, and 30 days ago
    data['lag_1'] = data[TARGET_COLUMN].shift(1)
    data['lag_7'] = data[TARGET_COLUMN].shift(7)
    data['lag_30'] = data[TARGET_COLUMN].shift(30)

    # 3. Rolling Window Features (e.g., 7-day rolling mean)
    data['rolling_mean_7'] = data[TARGET_COLUMN].shift(1).rolling(window=7).mean()

    return data

# Apply feature creation to the aggregated data
df_feat = create_time_series_features(df_agg.copy())

# 4. Handle Categorical Exogenous Variables (One-Hot Encoding for XGBoost)
df_feat = pd.get_dummies(df_feat, columns=['Weather Condition'], drop_first=True)

# Drop rows with NaN values resulting from lags/rolling windows
df_feat.dropna(inplace=True)

print("\nFeatures Created:")
print(df_feat.head().to_markdown(numalign="left", stralign="left"))


Features Created:
| Date                | Units Sold   | Price   | Discount   | Competitor Pricing   | Demand Forecast   | Holiday/Promotion   | dayofweek   | dayofyear   | weekofyear   | month   | year   | lag_1   | lag_7   | lag_30   | rolling_mean_7   | Weather Condition_Rainy   | Weather Condition_Snowy   | Weather Condition_Sunny   |
|:--------------------|:-------------|:--------|:-----------|:---------------------|:------------------|:--------------------|:------------|:------------|:-------------|:--------|:-------|:--------|:--------|:---------|:-----------------|:--------------------------|:--------------------------|:--------------------------|
| 2022-01-31 00:00:00 | 13187        | 55.2553 | 9.45       | 55.0981              | 136.82            | 1                   | 0           | 31          | 5            | 1       | 2022   | 13994   | 11277   | 14484    | 13284            | True                      | False                     | False                     |
| 2022-02-01

In [5]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.ensemble import RandomForestRegressor
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pickle
import plotly.express as px
import os
import matplotlib.pyplot as plt # Used for SARIMAX diagnostic plotting

# --- Configuration ---
file_name = "retail_store_inventory.csv"
TARGET_COLUMN = 'Units Sold'
TEST_SIZE = 30 # Number of days to use for validation/testing

# --- Helper Functions (KPIs & Feature Engineering) ---

def create_time_series_features(data):
    # This must match the features used in the training data exactly
    data['dayofweek'] = data.index.dayofweek
    data['dayofyear'] = data.index.dayofyear
    data['weekofyear'] = data.index.isocalendar().week.astype(int)
    data['month'] = data.index.month
    data['year'] = data.index.year
    data['lag_1'] = data[TARGET_COLUMN].shift(1)
    data['lag_7'] = data[TARGET_COLUMN].shift(7)
    data['lag_30'] = data[TARGET_COLUMN].shift(30)
    data['rolling_mean_7'] = data[TARGET_COLUMN].shift(1).rolling(window=7).mean()
    return data

def calculate_kpis(y_true, y_pred, model_name):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = mean_absolute_percentage_error(y_true, y_pred) * 100
    return {'RMSE': rmse, 'MAPE': mape}

# --- 1. Day 3: Modeling & Cross-Validation ---

# --- Data Preparation (Full Pipeline Re-run) ---

# Load and prepare data (same as Day 1/2)
df = pd.read_csv(file_name)
df['Date'] = pd.to_datetime(df['Date'])

# Aggregation and Exogenous Features
daily_sales = df.groupby('Date')[TARGET_COLUMN].sum().to_frame()
exog_numerical = df.groupby('Date')[['Price', 'Discount', 'Competitor Pricing', 'Demand Forecast']].mean()
exog_categorical = df.groupby('Date')[['Holiday/Promotion']].agg(lambda x: x.mode()[0])
df_agg = daily_sales.join([exog_numerical, exog_categorical])
df_agg.index.name = 'Date'
df_agg.fillna(method='ffill', inplace=True)

# Feature Engineering
df_feat = create_time_series_features(df_agg.copy())
df_feat.dropna(inplace=True)

# Data Split
train = df_feat.iloc[:-TEST_SIZE]
test = df_feat.iloc[-TEST_SIZE:]
FEATURES = df_feat.columns.drop(TARGET_COLUMN).tolist()
TARGET = TARGET_COLUMN

X_train, y_train = train[FEATURES], train[TARGET]
X_test, y_test = test[FEATURES], test[TARGET]


# --- A. Random Forest (ML Model) ---
print("--- Training Random Forest (Best Model) ---")
rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=10,
    random_state=42,
    n_jobs=-1
)
rf_model.fit(X_train, y_train)
rf_preds = rf_model.predict(X_test)
rf_kpis = calculate_kpis(y_test, rf_preds, "Random Forest")
print(f"Random Forest (Validation): RMSE={rf_kpis['RMSE']:.2f}, MAPE={rf_kpis['MAPE']:.2f}%")


# --- B. SARIMAX (Statistical Model) ---
print("\n--- Training SARIMAX (Baseline) ---")
SARIMAX_train = daily_sales.iloc[:-TEST_SIZE]
SARIMAX_test = daily_sales.iloc[-TEST_SIZE:]

try:
    sarimax_model = SARIMAX(
        SARIMAX_train[TARGET_COLUMN],
        order=(1, 1, 1),
        seasonal_order=(1, 1, 1, 7), # Weekly seasonality S=7
        enforce_stationarity=False,
        enforce_invertibility=False
    )
    sarimax_results = sarimax_model.fit(disp=False)
    sarimax_preds = sarimax_results.predict(
        start=SARIMAX_test.index[0],
        end=SARIMAX_test.index[-1]
    )
    sarimax_kpis = calculate_kpis(SARIMAX_test[TARGET_COLUMN], sarimax_preds, "SARIMAX")
    print(f"SARIMAX (Validation): RMSE={sarimax_kpis['RMSE']:.2f}, MAPE={sarimax_kpis['MAPE']:.2f}%")

    # Combine results
    sarimax_preds.index.name = 'Date'
    results_df = pd.DataFrame({
        'Actual Sales': y_test,
        'Random Forest Forecast': rf_preds,
        'SARIMAX Forecast': sarimax_preds
    })
except Exception as e:
    print(f"SARIMAX failed to converge: {e}. Only using Random Forest results.")
    results_df = pd.DataFrame({
        'Actual Sales': y_test,
        'Random Forest Forecast': rf_preds
    })




/tmp/ipython-input-612877599.py:50: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_agg.fillna(method='ffill', inplace=True)


--- Training Random Forest (Best Model) ---
Random Forest (Validation): RMSE=88.34, MAPE=0.53%

--- Training SARIMAX (Baseline) ---


/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


SARIMAX (Validation): RMSE=1233.44, MAPE=7.29%


In [6]:
# --- 2. Day 4: Model Deployment ---

# Train the final Random Forest model on ALL available feature-engineered data
print("\n--- Day 4: Final Model Training (Deployment) ---")

X_deploy = df_feat.drop(TARGET_COLUMN, axis=1)
y_deploy = df_feat[TARGET_COLUMN]

rf_deployment_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=10,
    random_state=42,
    n_jobs=-1
)
rf_deployment_model.fit(X_deploy, y_deploy)

# Export model as pickle
pickle_filename = 'rf_sales_forecast_model.pkl'
with open(pickle_filename, 'wb') as file:
    pickle.dump(rf_deployment_model, file)

print(f"Deployment Model Saved: {pickle_filename}")



--- Day 4: Final Model Training (Deployment) ---
Deployment Model Saved: rf_sales_forecast_model.pkl


In [7]:


# --- 3. Day 5: Delivery and Visualization ---

# Plotting the combined results
print("\n--- Day 5: Creating Final Plotly Dashboard ---")

fig = px.line(results_df, y=results_df.columns,
              title='Sales Forecast Comparison: Actuals vs. Models',
              labels={'value':'Units Sold', 'Date':'Date'},
              line_dash_map={'Actual Sales': 'solid', 'Random Forest Forecast': 'dot', 'SARIMAX Forecast': 'dash'}
)

fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Units Sold',
    hovermode='x unified',
    legend_title='Series'
)

# Save the visualization
plotly_filename = 'final_forecast_dashboard.json'
fig.write_json(plotly_filename)
print(f"Dashboard JSON Saved: {plotly_filename}")

# Save the final comparison data for review
results_df.to_csv('final_model_comparison_results.csv')
print("\nFinal comparison data saved to final_model_comparison_results.csv")


--- Day 5: Creating Final Plotly Dashboard ---
Dashboard JSON Saved: final_forecast_dashboard.json

Final comparison data saved to final_model_comparison_results.csv


In [8]:
import plotly.io as pio

# --- 1. Define the filename ---
plotly_filename = 'final_forecast_dashboard.json'

# --- 2. Load the Plotly figure from the JSON file ---
try:
    # Use pio.read_json() to deserialize the JSON file back into a Plotly Figure object
    fig = pio.read_json(plotly_filename)

    print(f"Plotly figure successfully loaded from {plotly_filename}")

    # --- 3. Display the interactive figure ---
    # In a Jupyter/IPython environment, fig.show() will render the interactive chart.
    fig.show()

except FileNotFoundError:
    print(f"Error: The file {plotly_filename} was not found. Please ensure it is in the current directory.")
except Exception as e:
    print(f"An error occurred while loading the plot: {e}")

Plotly figure successfully loaded from final_forecast_dashboard.json
